## 自由自举法

In [1]:
import breeze.linalg._
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.regression.LinearRegressionWithSGD
import org.apache.spark.mllib.regression.LinearRegressionModel
import org.apache.spark.mllib.optimization.{LBFGS,LeastSquaresGradient,SquaredL2Updater}
import org.apache.spark.mllib.linalg.Vectors
import breeze.numerics._
import breeze.stats.distributions._
import java.util.concurrent.ThreadLocalRandom
import org.apache.spark.mllib.util.MLUtils

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.176.1:4040
SparkContext available as 'sc' (version = 3.3.0, master = local[*], app id = local-1671708905892)
SparkSession available as 'spark'


import breeze.linalg._
import org.apache.spark.mllib.regression.LabeledPoint
import org.apache.spark.mllib.regression.LinearRegressionWithSGD
import org.apache.spark.mllib.regression.LinearRegressionModel
import org.apache.spark.mllib.optimization.{LBFGS, LeastSquaresGradient, SquaredL2Updater}
import org.apache.spark.mllib.linalg.Vectors
import breeze.numerics._
import breeze.stats.distributions._
import java.util.concurrent.ThreadLocalRandom
import org.apache.spark.mllib.util.MLUtils


In [2]:
val data=MLUtils.loadLibSVMFile(sc,"sample_linear_regression_data.txt")

data: org.apache.spark.rdd.RDD[org.apache.spark.mllib.regression.LabeledPoint] = MapPartitionsRDD[6] at map at MLUtils.scala:86


In [3]:
data.collect()(0) //这回的label是一个连续的变量了，第二个括号中的10是不为零的变量个数，第一个中括号为位置，第二个中括号为值

res0: org.apache.spark.mllib.regression.LabeledPoint = (-9.490009878824548,(10,[0,1,2,3,4,5,6,7,8,9],[0.4551273600657362,0.36644694351969087,-0.38256108933468047,-0.4458430198517267,0.33109790358914726,0.8067445293443565,-0.2624341731773887,-0.44850386111659524,-0.07269284838169332,0.5658035575800715]))


In [4]:
data.collect()(0).features

res1: org.apache.spark.mllib.linalg.Vector = (10,[0,1,2,3,4,5,6,7,8,9],[0.4551273600657362,0.36644694351969087,-0.38256108933468047,-0.4458430198517267,0.33109790358914726,0.8067445293443565,-0.2624341731773887,-0.44850386111659524,-0.07269284838169332,0.5658035575800715])


In [5]:
data.collect()(0).features.toArray

res2: Array[Double] = Array(0.4551273600657362, 0.36644694351969087, -0.38256108933468047, -0.4458430198517267, 0.33109790358914726, 0.8067445293443565, -0.2624341731773887, -0.44850386111659524, -0.07269284838169332, 0.5658035575800715)


In [6]:
val numFeatures=data.collect()(0).features.size

numFeatures: Int = 10


In [7]:
val training=data.map(x=> (x.label,MLUtils.appendBias(x.features))).cache()
training.take(1)(0)._2

22/12/22 19:35:19 WARN ProcfsMetricsGetter: Exception when trying to compute pagesize, as a result reporting of ProcessTree metrics is stopped


training: org.apache.spark.rdd.RDD[(Double, org.apache.spark.mllib.linalg.Vector)] = MapPartitionsRDD[7] at map at <console>:40
res3: org.apache.spark.mllib.linalg.Vector = (11,[0,1,2,3,4,5,6,7,8,9,10],[0.4551273600657362,0.36644694351969087,-0.38256108933468047,-0.4458430198517267,0.33109790358914726,0.8067445293443565,-0.2624341731773887,-0.44850386111659524,-0.07269284838169332,0.5658035575800715,1.0])


In [8]:
val training0=data.map(x=> LabeledPoint(x.label,MLUtils.appendBias(x.features))).cache()
training0.take(1)(0).features //与上对比，这个貌似更美丽

training0: org.apache.spark.rdd.RDD[org.apache.spark.mllib.regression.LabeledPoint] = MapPartitionsRDD[8] at map at <console>:40
res4: org.apache.spark.mllib.linalg.Vector = (11,[0,1,2,3,4,5,6,7,8,9,10],[0.4551273600657362,0.36644694351969087,-0.38256108933468047,-0.4458430198517267,0.33109790358914726,0.8067445293443565,-0.2624341731773887,-0.44850386111659524,-0.07269284838169332,0.5658035575800715,1.0])


In [9]:
val numCorrections = 10
val convergenceTol = 1e-4
val maxNumIterations = 20
val regParam=1.0
val initialWeightsWithIntercept=Vectors.dense(new Array[Double](numFeatures+1))

numCorrections: Int = 10
convergenceTol: Double = 1.0E-4
maxNumIterations: Int = 20
regParam: Double = 1.0
initialWeightsWithIntercept: org.apache.spark.mllib.linalg.Vector = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]


In [11]:
val (weightsWithIntercept0, loss) = LBFGS.runLBFGS(
training, 
new LeastSquaresGradient(),
new SquaredL2Updater(),
numCorrections,
convergenceTol,
maxNumIterations,
regParam,
initialWeightsWithIntercept)

//求得岭估计量
val coefficients = DenseVector(weightsWithIntercept0.toArray.slice(0, weightsWithIntercept0.size - 1))
val Intercept = weightsWithIntercept0(weightsWithIntercept0.size - 1)

//用有限内存BFGS估出betahat0

weightsWithIntercept0: org.apache.spark.mllib.linalg.Vector = [0.022917619144397203,0.16223861427526343,-0.1839044620018147,0.5454078186017856,0.10941974938462558,0.2811178300566573,-0.09271590011136743,-0.12330860139641511,-0.15505995633811184,0.16156545814727047,0.11614360033051634]
loss: Array[Double] = Array(53.156115128976595, 52.86141046393933, 52.801233318674036, 52.80046016202687, 52.8004497573181)
coefficients: breeze.linalg.DenseVector[Double] = DenseVector(0.022917619144397203, 0.16223861427526343, -0.1839044620018147, 0.5454078186017856, 0.10941974938462558, 0.2811178300566573, -0.09271590011136743, -0.12330860139641511, -0.15505995633811184, 0.16156545814727047)
Intercept: Double = 0.11614360033051634


In [12]:
var lines = data.map(line => {
    //val r = ThreadLocalRandom.current
    val fitted = DenseVector(line.features.toArray).t*coefficients + Intercept
    val residual = line.label - fitted  //epshat=y-yhat
    LabeledPoint(residual, line.features)  //关键：把残差放到label的位置上传出去
})
//计算残差

lines: org.apache.spark.rdd.RDD[org.apache.spark.mllib.regression.LabeledPoint] = MapPartitionsRDD[14] at map at <console>:42


In [ ]:
val transit = lines.map(line => {
    val r = ThreadLocalRandom.current
    val fitted = DenseVector(line.features.toArray).t*coefficients + Intercept
    LabeledPoint(fitted + r.nextGaussian * line.label, line.features) //继续这样传
})
//生成伪响应变量观测值

transit: org.apache.spark.rdd.RDD[org.apache.spark.mllib.regression.LabeledPoint] = MapPartitionsRDD[20] at map at <console>:76


In [ ]:
val training = transit.map(x => (x.label, MLUtils.appendBias(x.features))).cache() //加个截距

val (weightsWithIntercept, loss) = LBFGS.runLBFGS(
training, 
new LeastSquaresGradient(),
new SquaredL2Updater(),
numCorrections,
convergenceTol,
maxNumIterations,
regParam,
initialWeightsWithIntercept)
//再次进行估计

training: org.apache.spark.rdd.RDD[(Double, org.apache.spark.mllib.linalg.Vector)] = MapPartitionsRDD[21] at map at <console>:81
weightsWithIntercept: org.apache.spark.mllib.linalg.Vector = [0.004080738591034008,-0.36343661852356396,0.280388538151927,0.3124928731604595,-0.05369509192517111,0.2987967870284328,-0.12092415858519057,0.03734406242136414,-0.33946504538883904,0.2933085906829199,-0.09571720412163336]
loss: Array[Double] = Array(60.52557962124565, 60.13313564542861, 60.1011953238573, 60.10098411100185, 60.10096067990188)


## 子集合自举法

In [13]:
import scala.collection.JavaConverters._
import scala.util.Random
import org.apache.spark.rdd.PairRDDFunctions
import org.apache.spark.HashPartitioner

import scala.collection.JavaConverters._
import scala.util.Random
import org.apache.spark.rdd.PairRDDFunctions
import org.apache.spark.HashPartitioner


In [14]:
val n=2000000
val points=sc.parallelize(0 until n).map(iter => {
    val random=new Random()
    val u=random.nextDouble() 
    val mykey=if(u<0.5) 1 else 2  //将数据分成两份（两个key）
    (mykey,random.nextGaussian()) //键值对
}).partitionBy(new HashPartitioner(2)).persist()
//生成样本
//后面的partitionBy那些不需要知道

n: Int = 2000000
points: org.apache.spark.rdd.RDD[(Int, Double)] = ShuffledRDD[17] at partitionBy at <console>:52


In [15]:
val K=points.partitions.size  //key有几种，分成了多少类
val SampleFractions=List((1,0.004),(2,0.004)).toMap //想抽4000个

K: Int = 2
SampleFractions: scala.collection.immutable.Map[Int,Double] = Map(1 -> 0.004, 2 -> 0.004)


In [16]:
val SampledPoints=points.sampleByKey(false,SampleFractions)
println(SampledPoints.collect().size)
println(SampledPoints.partitions.size)

7895
2


SampledPoints: org.apache.spark.rdd.RDD[(Int, Double)] = MapPartitionsRDD[18] at sampleByKey at <console>:47


In [17]:
val N=SampledPoints.mapValues(x => 1).reduceByKey((x,y)=>x+y)
N.collect() //各个类别的样本量

N: org.apache.spark.rdd.RDD[(Int, Int)] = MapPartitionsRDD[20] at reduceByKey at <console>:46
res6: Array[(Int, Int)] = Array((2,3979), (1,3916))


In [18]:
//再对每个子集用自举法
val BootstrapFractions=List((1,1.0),(2,1.0)).toMap//抽100%
val BootstrappedPoints=SampledPoints.sampleByKeyExact(true,BootstrapFractions)
//有放回抽样
val EstBootSum=BootstrappedPoints.reduceByKey((x,y)=>x+y)
val EstBootMu=EstBootSum.join(N).mapValues(x=>x._1/x._2)//两类分别除以对应的N

BootstrapFractions: scala.collection.immutable.Map[Int,Double] = Map(1 -> 1.0, 2 -> 1.0)
BootstrappedPoints: org.apache.spark.rdd.RDD[(Int, Double)] = MapPartitionsRDD[24] at sampleByKeyExact at <console>:49
EstBootSum: org.apache.spark.rdd.RDD[(Int, Double)] = MapPartitionsRDD[25] at reduceByKey at <console>:51
EstBootMu: org.apache.spark.rdd.RDD[(Int, Double)] = MapPartitionsRDD[29] at mapValues at <console>:52


In [19]:
println(EstBootSum.join(N).collect()) //把N的两个值放在元组的后一个位置，且join by key
EstBootMu.collect() //仍然是rdd键值对

[Lscala.Tuple2;@1cb64f73


res7: Array[(Int, Double)] = Array((2,-0.020013682254099757), (1,-0.0271704819318725))


In [20]:
val UpdatedPoints=BootstrappedPoints.join(EstBootMu) //每个元素的后面跟一个自己的组均值
UpdatedPoints.take(3)

UpdatedPoints: org.apache.spark.rdd.RDD[(Int, (Double, Double))] = MapPartitionsRDD[35] at join at <console>:47
res8: Array[(Int, (Double, Double))] = Array((2,(-0.21069626728721133,-0.020013682254099757)), (2,(0.9529289320723532,-0.020013682254099757)), (2,(0.9529289320723532,-0.020013682254099757)))


In [21]:
val Est2thMomSum=UpdatedPoints.mapValues(x => (x._1-x._2)*(x._1-x._2)).reduceByKey((x,y)=> x+y)
//减均值再平方，再分别加起来
Est2thMomSum.collect()

Est2thMomSum: org.apache.spark.rdd.RDD[(Int, Double)] = MapPartitionsRDD[37] at reduceByKey at <console>:46
res9: Array[(Int, Double)] = Array((2,3639.8091180191363), (1,3688.2645726921533))


In [22]:
val Est2thMom=Est2thMomSum.join(N).mapValues(x => x._1/x._2)
Est2thMom.collect()

Est2thMom: org.apache.spark.rdd.RDD[(Int, Double)] = MapPartitionsRDD[41] at mapValues at <console>:47
res10: Array[(Int, Double)] = Array((2,0.9147547418997578), (1,0.9418448857742986))


In [23]:
//对k个估计再平均
val MetaEst2thMom=Est2thMom.values.reduce((x,y)=>x+y)/K
//使用values打破k的壁垒，再加起来

MetaEst2thMom: Double = 0.9282998138370282


In [24]:
Est2thMom.values.collect()

res11: Array[Double] = Array(0.9147547418997578, 0.9418448857742986)


In [25]:
Est2thMom.reduce((x,y) => (0,x._2+y._2))._2/K//这样一样

res12: Double = 0.9282998138370282


In [26]:
Est2thMom.values

res13: org.apache.spark.rdd.RDD[Double] = MapPartitionsRDD[44] at values at <console>:48


In [27]:
Est2thMom.map()

res14: org.apache.spark.rdd.RDD[(Int, Double)] = MapPartitionsRDD[41] at mapValues at <console>:47
